# Infrastructure Setup & Test

This notebook sets up and verifies all components of the Spend Categorization application.

## What This Notebook Does

1. **Spark Session** - Initialize Spark (works on Databricks or locally via Databricks Connect)
2. **Configuration** - Load and validate `config.yaml`
3. **Database Setup** - Initialize tables (prod mode) or verify MockBackend (test mode)
4. **Sample Data** - Optionally load sample invoices for testing
5. **Verification** - Test all query and correction functions

## Running This Notebook

- **Test Mode** (`mode: test` in config.yaml): Uses in-memory mock data, no external dependencies
- **Prod Mode** (`mode: prod` in config.yaml): Connects to Lakebase PostgreSQL


## 1. Spark Session Setup


In [ ]:
# Setup: Works both on Databricks and locally with Databricks Connect
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd() / "src"))

from utils import get_spark, is_running_on_databricks

spark = get_spark()
print(f"✅ Spark session initialized")
print(f"   Running on Databricks: {is_running_on_databricks()}")
print(f"   Spark version: {spark.version}")


## 2. Configuration Loading


In [ ]:
from invoice_app.config import load_config

config = load_config()

print(f"📋 Configuration Summary")
print(f"{'='*50}")
print(f"Mode:                    {config.app.mode}")
print(f"Is Test Mode:            {config.app.is_test_mode}")
print(f"Is Prod Mode:            {config.app.is_prod_mode}")
print(f"Invoices Table:          {config.tables.invoices}")
print(f"Corrections Table:       {config.tables.corrections}")
print(f"Page Size:               {config.app.page_size}")
print(f"Low Confidence Threshold: {config.flagging.low_confidence_threshold}")

if config.app.is_prod_mode:
    print(f"Lakebase Instance:       {config.app.lakebase_instance}")
    print(f"Lakebase Database:       {config.app.lakebase_dbname}")

print(f"{'='*50}")


## 3. Backend Initialization


In [ ]:
from invoice_app.database import (
    init_backend,
    get_backend,
    reset_backend,
    MockBackend,
    LakebaseBackend,
)

# Reset any existing backend state
reset_backend()

# Initialize the backend
backend = init_backend(config)

print(f"🔧 Backend Initialization")
print(f"{'='*50}")
print(f"Backend Type:            {type(backend).__name__}")
print(f"Is Connected:            {backend.is_connected()}")

if isinstance(backend, MockBackend):
    print(f"Mock Invoices Count:     {len(backend._invoices)}")
    print(f"Mock Categories Count:   {len(backend._categories)}")
    print(f"Mock Corrections Count:  {len(backend._corrections)}")
    print(f"\n✅ MockBackend ready for testing")
else:
    if backend.is_connected():
        print(f"\n✅ Connected to Lakebase")
    else:
        print(f"\n❌ Failed to connect to Lakebase")


## 4. Database Table Setup (Prod Mode Only)

This section creates the required tables in Lakebase. Skip this section in test mode.


In [ ]:
from invoice_app.corrections import (
    initialize_corrections_table,
    initialize_invoices_table,
)

if config.app.is_prod_mode and isinstance(backend, LakebaseBackend):
    print("🗄️ Initializing Database Tables")
    print(f"{'='*50}")
    
    try:
        # Create invoices table
        print(f"Creating table: {config.tables.invoices}...")
        initialize_invoices_table(config, backend)
        print(f"   ✅ Table {config.tables.invoices} created/verified")
        
        # Create corrections table
        print(f"Creating table: {config.tables.corrections}...")
        initialize_corrections_table(config, backend)
        print(f"   ✅ Table {config.tables.corrections} created/verified")
        
        print(f"\n✅ All tables initialized successfully")
    except Exception as e:
        print(f"\n❌ Failed to initialize tables: {e}")
else:
    print("⏭️ Skipping table initialization (test mode uses in-memory data)")


## 5. Load Sample Data (Optional)

Load sample invoices into the database for testing. Set `LOAD_SAMPLE_DATA = True` to enable.


In [ ]:
# Configuration
LOAD_SAMPLE_DATA = False  # Set to True to load sample data
SAMPLE_COUNT = 100        # Number of invoices to generate

if LOAD_SAMPLE_DATA and config.app.is_prod_mode:
    from utils import generate_sample_invoices
    
    print(f"📦 Loading {SAMPLE_COUNT} Sample Invoices")
    print(f"{'='*50}")
    
    try:
        invoices_df = generate_sample_invoices(SAMPLE_COUNT, seed=42)
        table = config.tables.invoices
        
        for i, row in invoices_df.iterrows():
            insert_query = f"""
                INSERT INTO {table}
                (invoice_id, invoice_number, transaction_id, vendor_name, 
                 invoice_date, amount, category, confidence_score, description)
                VALUES (:invoice_id, :invoice_number, :transaction_id, :vendor_name,
                        :invoice_date, :amount, :category, :confidence_score, :description)
                ON CONFLICT (invoice_id) DO NOTHING
            """
            backend.execute_write(insert_query, row.to_dict())
            
            if (i + 1) % 20 == 0:
                print(f"   Inserted {i + 1}/{SAMPLE_COUNT} invoices...")
        
        print(f"\n✅ Successfully loaded {SAMPLE_COUNT} sample invoices")
    except Exception as e:
        print(f"\n❌ Failed to load sample data: {e}")
elif LOAD_SAMPLE_DATA:
    print("⏭️ Skipping sample data load (test mode already has mock data)")
else:
    print("⏭️ Sample data loading disabled. Set LOAD_SAMPLE_DATA = True to enable.")


## 6. Query Functions Test

Test all query functions to ensure they work correctly with the backend.


## 7. Correction Functions Test

Test the Type 2 SCD correction functions.


In [ ]:
from invoice_app.corrections import (
    write_correction,
    write_corrections_batch,
)

print("✏️ Testing Correction Functions")
print(f"{'='*50}")

# Get initial correction count (for MockBackend)
if isinstance(backend, MockBackend):
    initial_count = len(backend._corrections)
    print(f"\nInitial corrections count: {initial_count}")

# Test 1: Write single correction
print(f"\n1. write_correction()")
write_correction(
    config,
    invoice_id="TEST-INV-001",
    transaction_ids=["TEST-TXN-001"],
    corrected_category="Hardware",
    comment="Infrastructure test correction",
    corrected_by="test_notebook",
    backend=backend,
)
print("   Single correction written")
print("   ✅ PASSED")

# Test 2: Write batch corrections
print(f"\n2. write_corrections_batch()")
batch_corrections = [
    {
        "invoice_id": "TEST-INV-002",
        "transaction_ids": ["TEST-TXN-002"],
        "corrected_category": "Software",
        "comment": "Batch test 1",
        "corrected_by": "test_notebook",
    },
    {
        "invoice_id": "TEST-INV-003",
        "transaction_ids": ["TEST-TXN-003"],
        "corrected_category": "Cloud Services",
        "comment": "Batch test 2",
        "corrected_by": "test_notebook",
    },
]
write_corrections_batch(config, batch_corrections, backend=backend)
print(f"   Batch of {len(batch_corrections)} corrections written")
print("   ✅ PASSED")

# Verify corrections were written (for MockBackend)
if isinstance(backend, MockBackend):
    final_count = len(backend._corrections)
    new_corrections = final_count - initial_count
    print(f"\n📊 Verification:")
    print(f"   New corrections added: {new_corrections}")
    assert new_corrections == 3, "Should have added 3 corrections"

print(f"\n{'='*50}")
print("✅ All correction functions working correctly")


## 8. Summary


In [ ]:
print("=" * 60)
print("🎉 INFRASTRUCTURE SETUP & TEST SUMMARY")
print("=" * 60)
print()
print(f"  Spark:             ✅ Initialized (v{spark.version})")
print(f"  On Databricks:     {is_running_on_databricks()}")
print(f"  Mode:              {config.app.mode.upper()}")
print(f"  Backend:           {type(backend).__name__}")
print(f"  Connected:         {backend.is_connected()}")
print(f"  Categories:        {len(categories)} available")
print(f"  Search Results:    {len(search_results)} found for 'Dell'")
print(f"  Flagged Invoices:  {len(flagged)} found")
print()
print("=" * 60)

if config.app.is_test_mode:
    print("""
✅ All tests passed in TEST mode!

Next steps to deploy to production:
1. Create a Lakebase instance in Databricks
2. Update config.yaml:
   - Set app.mode to 'prod'
   - Set app.lakebase_instance to your instance name
3. Re-run this notebook to initialize prod tables
4. Set LOAD_SAMPLE_DATA = True in section 5 to load test data
5. Run the app: uv run streamlit run app.py
""")
else:
    print("""
✅ All tests passed in PROD mode!

Your Lakebase infrastructure is ready. You can now:
1. Run the app: uv run streamlit run app.py
""")


In [ ]:
from invoice_app.queries import (
    search_invoices,
    get_flagged_invoices,
    get_invoices_by_ids,
    get_available_categories,
)

print("🔍 Testing Query Functions")
print(f"{'='*50}")

# Test 1: Get available categories
categories = get_available_categories(config, backend)
print(f"\n1. get_available_categories()")
print(f"   Categories found: {len(categories)}")
print(f"   Sample: {categories[:5]}")
assert len(categories) > 0, "Should have at least one category"
print("   ✅ PASSED")

# Test 2: Search invoices
search_results = search_invoices(config, "Dell", limit=10, backend=backend)
print(f"\n2. search_invoices('Dell')")
print(f"   Results found: {len(search_results)}")
if not search_results.empty:
    print(f"   Columns: {list(search_results.columns)}")
print("   ✅ PASSED")

# Test 3: Get flagged invoices
flagged = get_flagged_invoices(config, limit=10, backend=backend)
print(f"\n3. get_flagged_invoices()")
print(f"   Flagged invoices: {len(flagged)}")
if not flagged.empty:
    avg_confidence = flagged["confidence_score"].mean()
    print(f"   Avg confidence score: {avg_confidence:.2f}")
print("   ✅ PASSED")

# Test 4: Get invoices by IDs
if isinstance(backend, MockBackend):
    test_ids = backend._invoices["invoice_id"].head(3).tolist()
else:
    test_ids = search_results["invoice_id"].head(3).tolist() if not search_results.empty else []

if test_ids:
    invoices_by_id = get_invoices_by_ids(config, test_ids, backend=backend)
    print(f"\n4. get_invoices_by_ids({test_ids[:2]}...)")
    print(f"   Invoices retrieved: {len(invoices_by_id)}")
    assert len(invoices_by_id) == len(test_ids), "Should retrieve all requested invoices"
    print("   ✅ PASSED")
else:
    print("\n4. get_invoices_by_ids() - SKIPPED (no test IDs available)")

print(f"\n{'='*50}")
print("✅ All query functions working correctly")
